In [41]:
%%writefile mapper_wrod_to_group.py
#!/usr/bin/env python2
import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

with open('stop_words_en.txt', 'r') as stop_file:
    stop_words_str=stop_file.read()
stop_words_list = re.split('\n', stop_words_str, flags=re.UNICODE)

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    
    text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)    
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    
    for word in words:
        word = word.lower()
        if word in stop_words_list:
            print >> sys.stderr, "reporter:counter:Words stat,Stop words,%d" % 1
        else:
            key = ''.join(sorted(word))
            print "%s\t%s\t%d" % (key, word, 1)

Overwriting mapper_wrod_to_group.py


In [58]:
%%writefile reducer_groups.py
#!/usr/bin/env python2
import sys

current_key = None
current_word = None
current_word_sum = 0
isInit = True
current_words = {}

for line in sys.stdin:
    try:
        key, word, count = line.strip().split('\t', 2)
        count = int(count)
    except ValueError as e:
        continue
    
    print >> sys.stderr, "reporter:counter:Words stat,Word in reduce phase,%d" % 1
        
    if isInit:
        isInit = False
        current_key = key
        current_words[word] = 0
        current_words[word] += count
        continue

    if current_key != key:
        if current_key:
            total = sum(current_words.itervalues())
            unique = len(current_words.keys())

            if unique <= 1:
                print >> sys.stderr, "reporter:counter:Words stat,Single word,%d" % 1
            else:
                word_list = ','.join(sorted(current_words.keys()))
                print "%d\t%d\t%s" % (total, unique, word_list)
        current_words.clear()
        current_words[word] = 0
        current_key = key
    if word not in current_words:
        current_words[word] = 0
    current_words[word] += count

if current_key:
    total = sum(current_words.itervalues())
    unique = len(current_words.keys())

    if unique <= 1:
        print >> sys.stderr, "reporter:counter:Words stat,Single word,%d" % 1
    else:
        word_list = ','.join(sorted(current_words.keys()))
        print "%d\t%d\t%s" % (total, unique, word_list)


Overwriting reducer_groups.py


In [89]:
%%writefile reducer_final.py
#!/usr/bin/env python2
import sys
import re

for line in sys.stdin:
    try:
        total, unique, words = line.strip().split('\t', 2)
        if re.search(',english,|^english,|,english$', words):
            print line
    except ValueError as e:
        continue

Overwriting reducer_final.py


In [91]:
%%bash
GROUPS_DIR="groups"
RESULT_DIR="result"

hdfs dfs -rm -r -skipTrash ${RESULT_DIR} > /dev/null
hdfs dfs -rm -r -skipTrash ${GROUPS_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.job.name="Groups_count" \
    -D mapreduce.job.reduces=2 \
    -D stream.num.map.output.key.fields=2 \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
    -D mapreduce.partition.keycomparator.options="-k1,1 -k2,2" \
    -files mapper_wrod_to_group.py,reducer_groups.py,/datasets/stop_words_en.txt \
    -mapper "python mapper_wrod_to_group.py" \
    -reducer "python reducer_groups.py" \
    -input /data/wiki/en_articles_part \
    -output ${GROUPS_DIR} > /dev/null
    
yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.job.name="Find_english" \
    -D mapreduce.job.reduces=1 \
    -files reducer_final.py \
    -mapper "cat" \
    -reducer "python reducer_final.py" \
    -input ${GROUPS_DIR}/part-* \
    -output ${RESULT_DIR} > /dev/null    
        
hdfs dfs -cat ${RESULT_DIR}/part-* | head

7825	5	english,helsing,hesling,shengli,shingle
	


17/11/25 20:56:34 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/11/25 20:56:34 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/11/25 20:56:35 INFO mapred.FileInputFormat: Total input files to process : 1
17/11/25 20:56:35 INFO mapreduce.JobSubmitter: number of splits:2
17/11/25 20:56:35 INFO Configuration.deprecation: mapred.job.name is deprecated. Instead, use mapreduce.job.name
17/11/25 20:56:36 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1511597534846_0016
17/11/25 20:56:36 INFO impl.YarnClientImpl: Submitted application application_1511597534846_0016
17/11/25 20:56:36 INFO mapreduce.Job: The url to track the job: http://1502185b6101:8088/proxy/application_1511597534846_0016/
17/11/25 20:56:36 INFO mapreduce.Job: Running job: job_1511597534846_0016
17/11/25 20:56:42 INFO mapreduce.Job: Job job_1511597534846_0016 running in uber mode : false
17/11/25 20:56:42 INFO mapreduce.Job:  map 0% reduce 0%
17/11/25 20:56:59 INFO m